ТЕСТОВОЕ ЗАДАНИЕ для NLP инженера:

Цель: создать приложение на основе подхода Text Style Transfer (TST) и LLM

	0. Указать
		- планируемое время перед началом выполнения задачи в часах и минутах
		- фактически затраченное время на выполнение задачи в часах и минутах

	1. Создать парафразер (TST) на LLM
		- на вход получаем предложение в МУЖСКОМ роде
		- на выходе его перефразированный вариант в ЖЕНСКОМ роде + оценка качества перефразирования

	2. Подобрать модель - не более 7B
		- выбрать модель для задачи и обосновать свой выбор  (в лидерборде рассмотреть не менее 3 - выбор метрик и бенчмарков за автором (главное мочь его обосновать)

	3. Подобрать токенизатор
		- выбрать и обосновать свой выбор

	4. Файнтьюн выбранной модели 7B или более малой на любой домен, кроме Instruct

	5. Эксперимент с промтом (не менее 2 типов) CoT, CoT-SC, ToT, ReWOO, ReAct, APE, Auto-CoT, ART, другое
		- обосновать выбор (важно продемонстрировать один из перечисленных подходов, свой промт - вне лидерборда)

	5. Датасет
		- предложить методы эффективной предобработки под задачу TST
		- датасет приложить, либо ссылкой, либо .tar

	6. Подобрать и обосновать метрику для оценки близости предложения и его парафраза
		- Bleu
		- Rouge
		- Ter
		- Similarity
		- Meteor
		- Mera
		- Другое

	7. Создать интерфейс (gradio, streamlit, другое) для демонстрации работы отфайнтьюненной модели
		сценарий работы
		- на вход - предложение в МУЖСКОМ роде
		- на выход - предложение в ЖЕНСКОМ роде
    8. Опционально: добавить порог температуры генерации

# 0.Время

0. Время


планируемое время перед началом выполнения задачи в часах и минутах:
10:00

фактически затраченное время на выполнение задачи в часах и минутах: 10:00


# 1. Постановка задачи

Создать парафразер (TST) на LLM
    - на вход получаем предложение в МУЖСКОМ роде
    - на выходе его перефразированный вариант в ЖЕНСКОМ роде + оценка качества перефразирования

# установка библиотек

In [2]:
!pip install google-cloud-billing
!pip install fire
!pip install datasets
!pip install sacrebleu rouge-score sentence-transformers nltk
!pip install spacy
!python -m spacy download ru_core_news_sm
!pip install transformers
!pip install python-dotenv
!pip install boto3
!pip install -q bitsandbytes trl peft
!pip install -q transformers[torch] datasets
!pip install flash-attn --no-build-isolation
!pip install --upgrade torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 1.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha256=bb307e9e48f57d6d4093e5f567d453584113a81a7820769c91c30ed52f79b123
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver do

# Датасет

Сформируем 2 синтетических дата сета:

1) Testing dataset. Синтезируем его на наиболее старших платных моделях, в нашем случае, GeminiPro1.5. Провалидируем данные вручную.



2) Train /validation dataset . Синтезируем его на более быстрых и бюджетной модели Gemini
flash.


Необходимы файл .env для подключения к базе данных и .json c ключами от gemini.

Данные для обучения  сохранены в .tar файл
Данный блок следует запускать для увеличения объема данных либо изменения принципа их формирования.

## Подготовим подключение к базе данных, чтобы сохранять результаты генерации.

In [12]:
!python3.11 --version

Python 3.11.6


In [3]:
!pip install python-dotenv

/bin/bash: line 1: pip: command not found


In [4]:
# !pip install python-dotenv
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
# Step 1: Create the PostgreSQL directory for SSL certificates
!mkdir -p ~/.postgresql

# Step 2: Download the Yandex Cloud CA certificate using curl
!curl -o ~/.postgresql/root.crt "https://storage.yandexcloud.net/cloud-certs/CA.pem"

# Step 3: Set secure permissions on the downloaded certificate file
!chmod 0600 ~/.postgresql/root.crt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3579  100  3579    0     0  11981      0 --:--:-- --:--:-- --:--:-- 11969


In [ ]:
import psycopg2
import os
import numpy as np
import subprocess
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
def postgress_cloud_certs():
    # Define the target directory and file path
    target_dir = Path.home() / ".postgresql"
    file_path = target_dir / "root.crt"

    # Create the directory if it does not exist
    os.makedirs(target_dir, exist_ok=True)

    # Check if the file does not exist
    if not file_path.exists() :
        # Download the file using wget
        url = "https://storage.yandexcloud.net/cloud-certs/CA.pem"
        result = subprocess.run(["wget", url, "--output-document", str(file_path)])

        # Check if the download was successful
        if result.returncode == 0 :
            # Set the file permissions
            file_path.chmod(0o600)
            print(f"Downloaded and set permissions for {file_path}")
        else :
            print(f"Failed to download {url}")
    else :
        print(f"File {file_path} already exists")
def get_connection(dbname="db3"):
    postgress_user = os.environ.get("POSTGRESS_USER")
    postgress_pwd = os.environ.get("POSTGRESS_PWD")
    postgress_host = os.environ.get("POSTGRESS_HOST")
    try:
        postgress_cloud_certs()
    except:
        print("postgress_cloud_certs failed")
    return psycopg2.connect(f"""
        host={postgress_host}
        port=6432
        sslmode=verify-full
        dbname={dbname}
        user={postgress_user}
        password={postgress_pwd}
        target_session_attrs=read-write
    """)


In [ ]:
postgres_connection =get_connection(dbname="db3")

File /root/.postgresql/root.crt already exists


In [ ]:
table_name="SYNTH_BOY_GIRL"
cur =  postgres_connection.cursor()
create_new_table_querry=f"""
CREATE TABLE IF NOT EXISTS {table_name} (
  id SERIAL PRIMARY KEY,
  original TEXT,
  paraphrase TEXT,
  complexity TEXT,
  changes TEXT[],
  model TEXT,
  cost FLOAT,
  date TIMESTAMP
);"""
cur.execute(create_new_table_querry)
postgres_connection.commit()

In [ ]:

def download_table_as_dataframe (postgres_connection, table_name="SYNTH_BOY_GIRL") :
    """
    Downloads the contents of a specified table and returns it as a pandas DataFrame.

    :param table_name: str, the name of the table to download
    :return: pd.DataFrame, the contents of the table as a DataFrame
    """
    postgres_connection.rollback()
    cur = postgres_connection.cursor()
    # Query to select all data from the table
    query = f"SELECT * FROM {table_name};"
    cur.execute(query)

    # Fetch all data from the table
    rows = cur.fetchall()

    # Get column names
    colnames = [desc[0] for desc in cur.description]

    # Convert to DataFrame
    df = pd.DataFrame(rows, columns=colnames)

    cur.close()
    return df

In [ ]:
try:
  alter_query = f"ALTER TABLE {table_name} ADD COLUMN model TEXT"
  cur.execute(alter_query)
  postgres_connection.commit()
except:
  print("DuplicateColumn")

DuplicateColumn


## Генерируем тестовые данные и данные для обучения


### Подготовим классы для генерации и учета расходов на генерацию тестового набора данных  

In [ ]:
from google.cloud import billing_v1
from google.oauth2 import service_account
import pandas as pd
import os
import  itertools
from datetime import datetime

def get_prices():
    billing_client = billing_v1.CloudCatalogClient()

    # Initialize the nested dictionary


    g_models = ["Gemini 1.5 Pro", "Gemini 1.5 Flash"]
    token_lengths = ["longer than 128k tokens", "up to 128k tokens"]
    types = ["input", "output"]

    prices = {g_model : {token_length : {type : None for type in types} for token_length in token_lengths} for g_model in
              g_models}
    # Generate the Cartesian product
    conditions = list(itertools.product(g_models, token_lengths, types))

    billing_client = billing_v1.CloudCatalogClient()
    services = billing_client.list_services()
    for service in services :
        if "Gemini" in service.display_name:
            skus = billing_client.list_skus(parent=service.name)
            for sku in skus :
                for product, token_length, type in conditions :
                    if (product in sku.description) and (token_length in sku.description or  product=="Gemini Pro") and (type in sku.description.lower()) :
                        prices[product][token_length][type] = sku.pricing_info[0].pricing_expression.tiered_rates[0].unit_price.nanos / 1e9
    print(f"get prices {prices}")
    return  prices

class GeminiAccountant:
    prices = None  # Class attribute for prices

    @classmethod
    def initialize_prices (cls) :
        if cls.prices is None :
            cls.prices = get_prices()

    def __init__(self):
        self.initialize_prices()
        self.g_models = ["Gemini 1.5 Pro", "Gemini Pro", "Gemini 1.5 Flash"]
        self.token_lengths  = ["longer than 128k tokens", "up to 128k tokens"]
        self.types = ["input", "output"]
        self.model_dict = { "gemini-1.5-pro":"Gemini 1.5 Pro" ,
                            "gemini-1.5-flash":"Gemini 1.5 Flash",
                            "gemini-1.0-pro":"Gemini Pro"}

    def calculate_amount_gemini (self, input_tokens, output_tokens ,g_model="Gemini 1.5 Pro" ) :
        if self.model_dict[g_model] is not None:
            g_model=self.model_dict[g_model]
        if input_tokens <= 128000 :
            input_cost = input_tokens * self.prices[g_model]["up to 128k tokens"]["input"]
        else :
            input_cost = 128000 * self.prices[g_model]["up to 128k tokens"]["input"] + (
                        input_tokens - 128000) * self.prices[g_model]["longer than 128k tokens"]["input"]

        if output_tokens <= 128000 :
            output_cost = output_tokens * self.prices[g_model]["up to 128k tokens"]["output"]
        else :
            output_cost = 128000 * self.prices[g_model]["up to 128k tokens"]["output"] + (
                        output_tokens - 128000) * self.prices[g_model]["longer than 128k tokens"]["output"]

        total_cost = input_cost + output_cost
        return total_cost

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ='gemini_synt_data_set.json'

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel
import json
import os
import re
import pandas as pd
def get_project_id(full_path):
    # Extract project_id from the credentials file
    with open(full_path, 'r') as f :
        credentials = json.load(f)
        return credentials.get('project_id')
class Gemini:
    def __init__(self, model_name="gemini-1.5-pro",temperature=0):
        FULL_PATH = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
        project_id=get_project_id(FULL_PATH)
        vertexai.init(project=project_id, location="us-central1")
        self.model=GenerativeModel(model_name)
        self.model_name=model_name
        self.temperature=temperature
        self.generation_config= {
                "temperature" : self.temperature,
                'response_mime_type' : 'application/json'
            }
        self.accountant = GeminiAccountant()
    def _process_gemini_json_output (self , gemini_excerpts_str) :
        gemini_excerpts_str_1 = gemini_excerpts_str.replace('\n', '')
        gemini_excerpts_str_1 = gemini_excerpts_str_1.replace('json', '')
        gemini_excerpts_str_1 = gemini_excerpts_str_1.replace('```', '')
        return  gemini_excerpts_str_1

    def _save_to_postgress(self, postgres_connection ,result_df):
      cur =  postgres_connection.cursor()
      insert_query = f"""
                INSERT INTO {table_name} (original, paraphrase, complexity, changes, date , cost, model)
                VALUES (%s, %s, %s, %s, %s, %s, %s )
                """

      for index, row in result_df.iterrows() :
            cur.execute(insert_query, (
                row['original'],
                row['paraphrase'],
                row['complexity'],
                row['changes'],
                row['date'],
                row['cost'],
                row['model_name']
                ))
      result= postgres_connection.commit()
      cur.close()
      return result

    def infrence(self, request, postgres_connection=None ):
        date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        response = self.model.generate_content(request,generation_config=self.generation_config)
        text = response.candidates[0].content.parts[0].text
        text_cleaned = self._process_gemini_json_output(text)
        try :
            result_json = json.loads(text_cleaned)
            result_df=pd.DataFrame(data=result_json)
            result_df['date']=date

        except :
            print(f'json.loads does not work')

        cost=self.accountant.calculate_amount_gemini(input_tokens=response.usage_metadata.prompt_token_count,
                                                output_tokens=response.usage_metadata.candidates_token_count,
                                                g_model=self.model_name)
        result_df['cost']=cost
        result_df['model_name']=self.model_name
        return result_df


In [ ]:
gemini_1_5=Gemini(model_name="gemini-1.5-pro")
gemini_flash=Gemini(model_name="gemini-1.5-flash")

### Промпт для генерации.

In [ ]:
data_set_prompt="""Создай пример 10 примеров перефразирования мужского рода в женский.
Сформируй примеры из различных областей человеческого знания.
Сложные примеры могут включать предложения сложной структуры,
например деепричастые обороты и косвенную речь.
Сложные примеры могут состоять из нескольких предложений.
Необходимо использовать прошлое, будущее или настоящее время.
Некоторые примеры могут содержать косвенную речь.

Каждый пример состоит из:
"id": уникальный номер.
"original": Исходное предложение (в мужском роде)
"paraphrase":Парафраз (в женском роде)
"complexity": сложность которая может принимать значение "low","medium","high".
Метаданные (например, сложность трансформации, тип изменений)
результат необходимо представить следубщем формате:
[
  {
    "id": 1,
    "original": "Когда я увидел эту машину, я захотел ее купить.",
    "paraphrase": "Когда я увидела эту машину, я захотела ее купить.",
    "complexity": "low",
    "changes": ["subject", "verb"]
  },
  {
    "id": 2,
    "original": "Опытный врач внимательно осмотрит пациента и назначит лечение.",
    "paraphrase": "Опытная врач внимательно осмотрит пациента и назначит лечение.",
    "complexity": "medium",
    "changes": ["subject", "verb", "adjective"]
  },
  {
    "id": 3,
    "original": "Талантливый художник рисует картину, вдохновленный красотой природы. Мимо шел кот.
    -Пошел отсюда, крикнул он.",
    "paraphrase": "Талантливая художница рисует картину, вдохновленная красотой природы. Мимо шла кошка.
    -Пошла отсюда, крикнула она.",
    "complexity": "high",
    "changes": ["subject", "verb", "participle"]
  }
]"""

### Анализ результата генерации


In [ ]:
result_df = gemini_1_5.infrence(request=data_set_prompt,postgres_connection=postgres_connection)
result_df

,id,original,paraphrase,complexity,changes,date,cost,model_name
0,1,Молодой ученый представил свой доклад на конфе...,Молодая ученая представила свой доклад на конф...,low,"[subject, verb]",2024-09-04 11:53:25,0.011022,gemini-1.5-pro
1,2,Опытный врач внимательно осмотрит пациента и н...,Опытная врач внимательно осмотрит пациента и н...,medium,"[subject, adjective]",2024-09-04 11:53:25,0.011022,gemini-1.5-pro
2,3,"Талантливый художник рисует картину, вдохновле...","Талантливая художница рисует картину, вдохновл...",high,"[subject, verb, participle, pronoun]",2024-09-04 11:53:25,0.011022,gemini-1.5-pro
3,4,"Директор завода сообщил, что рабочий получил п...","Директор завода сообщила, что работница получи...",medium,"[subject, verb, object]",2024-09-04 11:53:25,0.011022,gemini-1.5-pro
4,5,"Известный писатель закончил свой новый роман, ...",Известная писательница закончила свой новый ро...,medium,"[subject, verb, pronoun]",2024-09-04 11:53:25,0.011022,gemini-1.5-pro
5,6,"По словам очевидцев, мужчина в сером пальто уб...","По словам очевидцев, женщина в сером пальто уб...",medium,[object],2024-09-04 11:53:25,0.011022,gemini-1.5-pro
6,7,"Мальчик мечтал стать космонавтом, как Юрий Гаг...","Девочка мечтала стать космонавтом, как Валенти...",high,"[subject, proper name]",2024-09-04 11:53:25,0.011022,gemini-1.5-pro
7,8,Старый учитель рассказывал своим ученикам о по...,Старая учительница рассказывала своим ученикам...,medium,"[subject, verb, adjective]",2024-09-04 11:53:25,0.011022,gemini-1.5-pro
8,9,Продавец-консультант помог покупателю выбрать ...,Продавец-консультант помогла покупательнице вы...,medium,"[subject, verb, object]",2024-09-04 11:53:25,0.011022,gemini-1.5-pro
9,10,"Водитель автобуса, рискуя жизнью, вывел пассаж...","Водитель автобуса, рискуя жизнью, вывела пасса...",medium,"[subject, verb]",2024-09-04 11:53:25,0.011022,gemini-1.5-pro


In [ ]:
result_df = gemini_flash.infrence(request=data_set_prompt,postgres_connection=postgres_connection)
result_df_flash

,id,original,paraphrase,complexity,changes,date,cost,model_name
0,1,Молодой ученый представил свой доклад на конфе...,Молодая ученая представила свой доклад на конф...,low,"[subject, verb]",2024-09-04 12:01:42,0.000305,gemini-1.5-flash
1,2,Опытный врач внимательно осмотрит пациента и н...,Опытная врач внимательно осмотрит пациента и н...,medium,"[subject, verb, adjective]",2024-09-04 12:01:42,0.000305,gemini-1.5-flash
2,3,"Талантливый художник рисует картину, вдохновле...","Талантливая художница рисует картину, вдохновл...",high,"[subject, verb, participle]",2024-09-04 12:01:42,0.000305,gemini-1.5-flash
3,4,"Известный писатель написал роман, который стал...","Известная писательница написала роман, который...",low,"[subject, verb]",2024-09-04 12:01:42,0.000305,gemini-1.5-flash
4,5,Профессор университета читал лекцию по истории...,Профессор университета читала лекцию по истори...,low,"[subject, verb]",2024-09-04 12:01:42,0.000305,gemini-1.5-flash
5,6,"Строитель возводил дом, используя современные ...","Строительница возводила дом, используя совреме...",low,"[subject, verb]",2024-09-04 12:01:42,0.000305,gemini-1.5-flash
6,7,"Полицейский остановил машину, нарушившую прави...","Полицейская остановила машину, нарушившую прав...",low,"[subject, verb]",2024-09-04 12:01:42,0.000305,gemini-1.5-flash
7,8,"Директор компании представил новый проект, кот...","Директор компании представила новый проект, ко...",low,"[subject, verb]",2024-09-04 12:01:42,0.000305,gemini-1.5-flash
8,9,"Спортсмен тренировался упорно, готовясь к соре...","Спортсменка тренировалась упорно, готовясь к с...",low,"[subject, verb]",2024-09-04 12:01:42,0.000305,gemini-1.5-flash
9,10,"Музыкант сочинил новую мелодию, вдохновленный ...","Музыкантка сочинила новую мелодию, вдохновленн...",low,"[subject, verb]",2024-09-04 12:01:42,0.000305,gemini-1.5-flash


10 примеров на gemini-pro1.5 стоят около 0.01 доллара.  1 пример занимет 1,5 секунды. Можно сделать многопоточность и повысить скорость генерации при необходимости.
Для test data set возьмем около 100 примеров.

10 примеров на Gemini 1.5 Flash стоят около 0.0003 доллара.  1 пример занимет 0.6 секунды.
Для train validation set возьмем для начала 1000 примеров.


### Генерация неоходимого объема данных

In [ ]:
# for i in range(10):
#     result_df = gemini_1_5.infrence(request=data_set_prompt,postgres_connection=postgres_connection)
#     postgres_connection.rollback()
#     gemini_1_5._save_to_postgress(postgres_connection ,result_df)

In [ ]:
# for i in range(100):
#     result_df = gemini_flash.infrence(request=data_set_prompt,postgres_connection=postgres_connection)
#     postgres_connection.rollback()
#     gemini_flash._save_to_postgress(postgres_connection ,result_df)

## Получение данных из базы


In [ ]:
data_df=download_table_as_dataframe (postgres_connection)

In [ ]:
testdf=data_df[data_df['model']=='gemini-1.5-pro']

In [ ]:
traindf=data_df[data_df['model']!='gemini-1.5-pro']

In [ ]:
traindf.to_csv('traindf.csv')

In [ ]:
traindf.shape

(1000, 8)

# 2. Подбор модели

## Ограничения:


Не более 7B

## Выбор моделей:  


Рассмотрим лучшие модели на лидербордах:

https://lmarena.ai/

Выбираем русский язык и opensource варианты, которые есть.

ограничению (не больше 7b и не Instruct) соотвествуют:

- Gemma-2-2b-it
- Llama-2-7b-chat
- Gemma-7b-it
- Mistral-7B-v0.1

так же добавим небольшую но хорошо обученную на русском языке модель
- FRED-T5-1.7B

In [ ]:
import torch
from transformers import GPT2Tokenizer, T5ForConditionalGeneration

In [ ]:

# tokenizer = GPT2Tokenizer.from_pretrained('ai-forever/FRED-T5-1.7B',eos_token='</s>')
# model = T5ForConditionalGeneration.from_pretrained('ai-forever/FRED-T5-1.7B')
# device='cuda'
# model.to(device)

In [ ]:
lm_text='<LM>род должен быть женским '


In [ ]:
# input_ids=torch.tensor([tokenizer.encode(lm_text)]).to(device)
# outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id,early_stopping=False)
# print(tokenizer.decode(outputs[0][1:]))

## Файнтюн Выбранных моделей.


### Подготовим basket, куда будем сохранять модель

In [ ]:
import boto3
import pandas as pd
import os
from botocore.client import Config
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

# Yandex Object Storage endpoint
endpoint_url = 'https://storage.yandexcloud.net'

# Yandex credentials

access_key = os.environ.get("YANDEX_KEY_ID")
secret_key = os.environ.get("YANDEX_KEY")

# Initialize a session using Yandex Object Storage
session = boto3.session.Session()
s3_client = session.client(
    service_name='s3',
    endpoint_url=endpoint_url,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4')  # Specify the signature version
)


In [ ]:
s3_client.upload_file("traindf.csv", "alex-llm-models", "traindf.csv")

### LLAMA fine tune

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 8.0 MB/s eta 0:00:00


In [ ]:
from typing import List
import fire
import torch
import transformers
from datasets import Dataset
import pandas as pd
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PrefixTuningConfig,
    TaskType
)
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
import boto3
import shutil
from trl import SFTTrainer

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [ ]:
base_model = "mistralai/Mistral-7B-v0.1"
# base_model = "meta-llama/Llama-2-7b-hf"

In [ ]:
output_dir=base_model

In [ ]:
micro_batch_size: int = 4,
gradient_accumulation_steps: int = 4,
num_epochs: int = 3,
learning_rate: float = 3e-4,
val_set_size: int = 200,

# lora hyperparams
lora_r: int = 8,
lora_alpha: int = 16,
lora_dropout: float = 0.05,
lora_target_modules: List[str] = [
  "q_proj",
  "v_proj",
]

In [ ]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=128,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=[
        "q_proj",
        "v_proj",
    ],
)

In [ ]:
model = get_peft_model(model, config)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
dataset = Dataset.from_pandas(traindf)

In [ ]:
def tokenize(data):
    source_ids = tokenizer.encode(data['original'], truncation=True, padding='max_length', max_length=512)
    target_ids = tokenizer.encode(data['paraphrase'], truncation=True, padding='max_length', max_length=512)

    input_ids = source_ids + target_ids + [tokenizer.eos_token_id]
    labels = [-100] * len(source_ids) + target_ids + [tokenizer.eos_token_id]

    return {
        "input_ids": input_ids,
        "labels": labels
    }

In [ ]:
train_val = dataset.train_test_split(seed=42)
train_data = train_val["train"].apply(tokenize, axis=1)
val_data = train_val["test"].shuffle().map(tokenize)

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:

# Step 2: Define the Tokenization Function
def tokenize(row):
    # Tokenize the source text (original)
    source = tokenizer(
        row['original'],
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )

    # Tokenize the target text (paraphrase)
    target = tokenizer(
        row['paraphrase'],
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )

    # Prepare input_ids by concatenating source and target with eos_token
    input_ids = torch.cat([source['input_ids'], target['input_ids']], dim=-1).squeeze()

    # Prepare labels to ignore padding tokens in the source part
    labels = torch.cat(
        [torch.full(source['input_ids'].shape, -100), target['input_ids']],
        dim=-1
    ).squeeze()

    return {
        "input_ids": input_ids.tolist(),
        "labels": labels.tolist()
    }


In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoTokenizer, TrainingArguments, AutoModelForCausalLM
from trl import SFTTrainer
from sklearn.model_selection import train_test_split


# Step 4: Split the Data into Training and Evaluation Datasets
train_df, eval_df = train_test_split(traindf, test_size=0.2, random_state=42)

# Step 5: Tokenize the DataFrames
train_tokenized = train_df.apply(tokenize, axis=1)
eval_tokenized = eval_df.apply(tokenize, axis=1)

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(pd.DataFrame(train_tokenized.tolist()))
eval_dataset = Dataset.from_pandas(pd.DataFrame(eval_tokenized.tolist()))


In [ ]:
# Step 6: Define the Formatting Function
def formatting_func(examples):
    # Concatenate original and paraphrase with a separator token
    formatted_texts = examples['original'] + " <|endoftext|> " + examples['paraphrase']
    return formatted_texts


In [ ]:
!pip install torch==2.2.0

  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB ? eta 0:00:00
Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB ? eta 0:00:00
Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 499.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 535.5 kB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.0.0
    Uninstalling triton-3.0.0:
      Successfully uninstalled triton-3.0

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",        # Save at the end of each epoch
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,  # Ensure that the best model is loaded at the end
    fp16=True,  # Enable mixed precision training
)


# Step 8: Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    formatting_func=formatting_func,
    packing=False
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:509: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a valid formatting function. Therefore `formatting_func` will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:407: UserWarning: Y

NotImplementedError: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.

In [1]:
def get_device_map() -> str:
    return 'cuda' if torch.cuda.is_available() else 'cpu'


device = get_device_map()  # 'cpu'

NameError: name 'torch' is not defined

In [ ]:
device

'cuda'

In [ ]:

from trl import SFTTrainer, SFTConfig

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=1,  # Adjust based on GPU memory
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,  # Increase to effectively increase batch size
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    fp16=True,  # Enable mixed precision for faster training
)

# Метрики

## Оценним работу метрик на нашем тестовом дата сете

### BLUE

In [ ]:
from nltk.translate.meteor_score import meteor_score
from sentence_transformers import SentenceTransformer, util
from sacrebleu import corpus_bleu
from rouge_score import rouge_scorer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
originals = test_df['original'][0]
paraphrases = test_df['paraphrase'][0]


In [ ]:
originals,paraphrases

('Известный писатель опубликовал новый роман.',
 'Известная писательница опубликовала новый роман.')

In [ ]:
bleu_score = corpus_bleu(paraphrases,originals)
bleu_score

BLEU = 0.00 100.0/0.0/0.0/0.0 (BP = 1.000 ratio = 1.000 hyp_len = 1 ref_len = 1)

In [ ]:
originals = test_df['original'][0]
paraphrases = test_df['paraphrase'][21]

In [ ]:
originals,paraphrases

('Известный писатель опубликовал новый роман.',
 'Опытная врач внимательно осмотрит пациента и назначит лечение.')

In [ ]:
bleu_score = corpus_bleu(paraphrases,originals)

In [ ]:
bleu_score

BLEU = 0.00 0.0/0.0/0.0/0.0 (BP = 1.000 ratio = 1.000 hyp_len = 1 ref_len = 1)

 Метрика BLEU показывает схожесть грамм, но для нее писатель и писательница - это разные слова. Нам нужна метрика, которая не будет нас штрафовать за смету рода.

### METEOR SPACY ru_core_news_sm

In [ ]:

nlp = spacy.load('ru_core_news_sm')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 53.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from transformers import AutoTokenizer
from nltk.translate.meteor_score import meteor_score
import nltk

# Загрузка ресурса WordNet
nltk.download('wordnet')
nltk.download('omw-1.4')  # Загрузка дополнительных данных WordNet

# Теперь можно использовать meteor_score
from nltk.translate.meteor_score import meteor_score

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:

original = test_df['original'][0]
paraphrase = test_df['paraphrase'][0]
# Токенизация предложений с использованием SpaCy
original_tokens = [token.text for token in nlp(original)]
paraphrase_tokens = [token.text for token in nlp(paraphrase)]
meteor = meteor_score([original_tokens], paraphrase_tokens)

print(f"original = {original}\nparaphrase ={paraphrase}\nMETEOR Score: {meteor:.4f}")

original = Известный писатель опубликовал новый роман.
paraphrase =Известная писательница опубликовала новый роман.
METEOR Score: 0.4907


In [ ]:
original = test_df['original'][0]
paraphrase = test_df['paraphrase'][21]
original_tokens = [token.text for token in nlp(original)]
paraphrase_tokens = [token.text for token in nlp(paraphrase)]
meteor = meteor_score([original_tokens], paraphrase_tokens)
print(f"original = {original}\nparaphrase ={paraphrase}\nMETEOR Score: {meteor:.4f}")

original = Известный писатель опубликовал новый роман.
paraphrase =Опытная врач внимательно осмотрит пациента и назначит лечение.
METEOR Score: 0.0794


Метеор подходит, чтобы отличать более схожий текст, качество зависит от токенизатора.

### METEOR + bert-base-multilingual-cased tokenizer

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
original = test_df['original'][0]
paraphrase = test_df['paraphrase'][0]
original_tokens = tokenizer.tokenize(original)
paraphrase_tokens = tokenizer.tokenize(paraphrase)
meteor = meteor_score([original_tokens], paraphrase_tokens)
print(f"original = {original}\nparaphrase ={paraphrase}\nMETEOR Score: {meteor:.4f}")

original = Известный писатель опубликовал новый роман.
paraphrase =Известная писательница опубликовала новый роман.
METEOR Score: 0.8152


In [ ]:
original = test_df['original'][0]
paraphrase = test_df['paraphrase'][21]
original_tokens = tokenizer.tokenize(original)
paraphrase_tokens = tokenizer.tokenize(paraphrase)
meteor = meteor_score([original_tokens], paraphrase_tokens)
print(f"original = {original}\nparaphrase ={paraphrase}\nMETEOR Score: {meteor:.4f}")

original = Известный писатель опубликовал новый роман.
paraphrase =Опытная врач внимательно осмотрит пациента и назначит лечение.
METEOR Score: 0.0495


METEOR + bert-base-multilingual-cased tokenizer гораздо четче показывает разницу между более схожим и сильно отличающимся текстом.

### cosine_similarity + google embedding model.

In [ ]:
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel

In [ ]:
embedding_model = TextEmbeddingModel.from_pretrained( "text-embedding-004")
original = test_df['original'][0]
paraphrase = test_df['paraphrase'][0]
original_embedding =  embedding_model.get_embeddings([original])
original_embedding= [embedding.values for embedding in original_embedding]
paraphrase_embedding = embedding_model.get_embeddings([paraphrase])
paraphrase_embedding= [embedding.values for embedding in paraphrase_embedding]
np_original_embedding = np.array(original_embedding)
np_paraphrase_embedding = np.array(paraphrase_embedding)
cs_scores = cosine_similarity(np_original_embedding, np_paraphrase_embedding)
print(f"original = {original}\nparaphrase ={paraphrase}\ncs_scores: {cs_scores[0][0]:.4f}")

original = Известный писатель опубликовал новый роман.
paraphrase =Известная писательница опубликовала новый роман.
cs_scores: 0.9798


In [ ]:
embedding_model = TextEmbeddingModel.from_pretrained( "text-embedding-004")
original = test_df['original'][0]
paraphrase = test_df['paraphrase'][21]
original_embedding =  embedding_model.get_embeddings([original])
original_embedding= [embedding.values for embedding in original_embedding]
paraphrase_embedding = embedding_model.get_embeddings([paraphrase])
paraphrase_embedding= [embedding.values for embedding in paraphrase_embedding]
np_original_embedding = np.array(original_embedding)
np_paraphrase_embedding = np.array(paraphrase_embedding)
cs_scores = cosine_similarity(np_original_embedding, np_paraphrase_embedding)
print(f"original = {original}\nparaphrase ={paraphrase}\ncs_scores: {cs_scores[0][0]:.4f}")

original = Известный писатель опубликовал новый роман.
paraphrase =Опытная врач внимательно осмотрит пациента и назначит лечение.
cs_scores: 0.4910


косинусная близость - так же достаточно репрезентативная метрика.

### Вывод: METEOR + bert-base-multilingual-cased tokenizer показывают хороший результат - метрика низкая для сильно различающихся предложений и высокая для очень похожих. Кроме того ее использование не требует GPU или платных API.